<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/en_sentence_complete_0912.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영어 교정 Transformer

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers
!pip3 install sentence-transformers

     |████████████████████████████████| 2.8 MB 7.5 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 3.3 MB 63.2 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 85 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=1076e747bf3c4c614edaef20480319cfa23c79f100d70fee5ca724ef32570eef
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [3]:
import matplotlib.pyplot as plt
# set seeds for reproducability
from numpy.random import seed
#seed(1)

import pandas as pd
import numpy as np
import string, os 

import urllib.request
import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
import tensorflow_datasets as tfds
train_data, test_data = tfds.load(name="cnn_dailymail",split=(tfds.Split.TRAIN,tfds.Split.TEST),with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteCZ03CK/cnn_dailymail-train.tfrecord


...

  0%|          | 0/13368 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteCZ03CK/cnn_dailymail-test.tfrecord


  0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [23]:
iterator = iter(train_data[0])

In [24]:
sentences_dataset = []

for data in iterator:
    news = data['article'].numpy().decode('UTF-8')
    sentences = np.array(nltk.sent_tokenize(news))
    for sen in sentences:
        if len(sen) > 30:
            sentences_dataset.append(sen)
    if len(sentences_dataset) > 1000000:
        break

In [25]:
len(sentences_dataset)

1000001

In [28]:
sentences_dataset[0:100]

['The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.',
 'The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.',
 'Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A .',
 "State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure.",
 'The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A.',
 'The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops in Italy last month.',
 'Symptoms of hepatitis A include fever, tiredness, loss o

In [29]:
import random

def crush_sent(txt,crush_rate=0.3):
    sens = np.array(txt.split(' '))
    l = len(sens)
    r = int(l*crush_rate)
    s = [i for i in range(l)]
    c = random.sample(s,r)
    t = [x for x in s if (x not in c)]
    crushed_text = ' '.join(sens[t])
    score = 0 #grammer_checker.transfer_learning([crushed_text])[1][0,1].item()
    return crushed_text,score,txt

In [30]:
txt="Targeting Russia's peripatetic business community would be one way of sapping their tenuous support for President Putin."
crush_sent(txt)

('Targeting peripatetic business community would be one way sapping their tenuous President',
 0,
 "Targeting Russia's peripatetic business community would be one way of sapping their tenuous support for President Putin.")

In [31]:
random.shuffle(sentences_dataset)

In [32]:
import pickle

dataset = []
for i,sentence in enumerate(sentences_dataset):
    print(f'\r {i+1}/{len(sentences_dataset)}', end="", flush=True)
    src,score,trg = crush_sent(sentence)
    dataset.append((src,score,trg))
    if i%5000 == 0:
        with open("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_dataset.bin", "wb") as fp:   #Pickling
            pickle.dump(dataset, fp)

 1000001/1000001

In [33]:
len(dataset)

1000001

In [34]:
pre_trained_kobert_model_name='bert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:

from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pre_trained_kobert_model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [36]:
op = tokenizer("Targeting Russia's peripatetic business community would be one way of sapping their tenuous support for President Putin.", return_tensors="pt",padding="max_length", truncation=True, max_length=64)
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[  101, 14126,  3607,  1005,  1055,  2566, 11514,  3686,  4588,  2449,
          2451,  2052,  2022,  2028,  2126,  1997, 20066,  4691,  2037,  2702,
          8918,  2490,  2005,  2343, 22072,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [37]:
curren_sentence_num = 0
dataset_iterator = []

In [38]:
encoder_max_length = 128
decoder_max_length = 128
batch_size = 16 #4 # 64, 128

for cnt in range(curren_sentence_num,len(dataset),batch_size):
    print(f'\r {cnt+1}/{len(dataset)}', end="", flush=True)
    batch = {}
    batch['input_ids'] = []
    batch['attention_mask'] = []
    batch['decoder_input_ids'] = []
    batch['decoder_attention_mask'] = []
    batch['labels'] = []

    for i in range(cnt,cnt+batch_size):
        (src,score,trg) = dataset[i]
        inputs = tokenizer(src.lower(), padding="max_length", truncation=True, max_length=encoder_max_length)
        outputs = tokenizer(trg.lower(), padding="max_length", truncation=True, max_length=decoder_max_length)
        batch["input_ids"].append(inputs.input_ids)
        batch["attention_mask"].append(inputs.attention_mask)
        batch["decoder_input_ids"].append(outputs.input_ids)
        batch["decoder_attention_mask"].append(outputs.attention_mask)
        batch["labels"].append(outputs.input_ids.copy())
    
    dataset_iterator.append(batch)
    if cnt%50000 == 0:
        with open("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_tokenized_dataset.bin", "wb") as fp:   #Pickling
            pickle.dump(dataset_iterator, fp)
            print('저장')

 1/1000001저장
 50001/1000001저장
 100001/1000001저장
 150001/1000001저장
 200001/1000001저장
 250001/1000001저장
 300001/1000001저장
 350001/1000001저장
 400001/1000001저장
 450001/1000001저장
 500001/1000001저장
 550001/1000001저장
 600001/1000001저장
 650001/1000001저장
 700001/1000001저장
 750001/1000001저장
 800001/1000001저장
 850001/1000001저장
 900001/1000001저장
 950001/1000001저장
 1000001/1000001

IndexError: ignored

In [39]:
len(dataset_iterator)

62500

In [40]:
with open("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_tokenized_dataset.bin", "wb") as fp:   #Pickling
    pickle.dump(dataset_iterator, fp)

In [41]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def generate_summary(text):
    # cut off at BERT max length 512
    sens = nltk.sent_tokenize(text)
    assert(len(sens) == 2)
    inputs = tokenizer(sens[0].strip()+'[SEP]'+sens[1].strip(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

def generate_summary2(text):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask).cpu().detach().numpy()[0]
    o=[]
    for token in outputs:
        if token == tokenizer.pad_token_id:
            break
        o.append(token)
    output_str = tokenizer.batch_decode([o], skip_special_tokens=True)[0]

    return output_str

In [42]:
from transformers import EncoderDecoderModel, BertTokenizerFast

try:
    del model
    print('delete model')
except Exception as ex:
    pass

use_pretrained_model = False

if use_pretrained_model:
    model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_model")
    print('Load from pre-trained model...')
else:
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(pre_trained_kobert_model_name, pre_trained_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints
    print('Initialize with ',pre_trained_kobert_model_name)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

N_EPOCHS = 1

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

Initialize with  bert-base-uncased


In [43]:
from transformers import BertTokenizer, AutoTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
        
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)

In [44]:
def train2(model, iterator, optimizer, criterion, scheduler,clip):
    
    model.train()
    
    epoch_loss = 0
    optimizer.zero_grad()
    #for i, batch in enumerate(iterator):
    for i in range(0,len(iterator)):
        batch = iterator[i]
        print(f'\r batch {i+1}/{len(iterator)}', end="", flush=True)

        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        decoder_input_ids= torch.tensor(batch["decoder_input_ids"]).to(device)
        decoder_attention_mask= torch.tensor(batch["decoder_attention_mask"]).to(device)
        labels= torch.tensor(batch["labels"]).to(device)

            
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)

        loss = outputs.loss #* b_rewards
        loss.backward()
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        scheduler.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)

        if i%1000 == 0:
            print('\tTarget text:',generate_summary2('BERT is transformers model pretrained on a large of English in self-supervised fashion.'))      
            model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_model")
            model.train()
            
    return epoch_loss / len(iterator)

BERT is a transformers model pretrained on a large corpus of English data in a self-supervised fashion.

BERT is transformers model pretrained on a large of English in self-supervised fashion.

In [ ]:
import time
import random
import datetime
import math

CLIP = 1

best_valid_loss = float('inf')
model.to(device)

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(dataset_iterator) * N_EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train2(model, dataset_iterator, optimizer, criterion, scheduler,CLIP)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

    doc_end_time = time.time()
    model.save_pretrained("/content/drive/MyDrive/GAN_ENDE/en_sentence_complete_model")